In [2]:
#Importing relevant libraries
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Connecting to the database
conn_str = ('DRIVER={ODBC Driver 17 for SQL Server};Server=analiza.database.windows.net;Database=AnalizaDB;UID=qa3107;PWD=fA8r2J2_DZH<;')
conn = pyodbc.connect(conn_str)

In [ ]:
#Data Collection And Loading
datasets_names= ['Products','Suppliers','[Order Details]','Categories','Territories','Region','EmployeeTerritories','Employees','Orders','Shippers','Customers']
dataframes = [] #a list of all dataframes loaded, later used for quick review of all datasets loaded. 
def get_dataset(dataset):
    dataset = pd.read_sql(f"SELECT * FROM Northwind.{dataset}", conn)
    return dataset

for dataset in datasets_names:
    dataframe = get_dataset(dataset)
    dataframe.name = f'{dataset}' #Adding a name for each dataframe for later readability
    dataframes.append(dataframe) 

conn.close

C:\Users\ys170\AppData\Local\Temp\ipykernel_12668\1573331519.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataset = pd.read_sql(f"SELECT * FROM Northwind.{dataset}", conn)


<function Connection.close()>

In [38]:
#Making sure all data was loaded properly

#looking for 11 dataframes 
print(len(dataframes)) 
for df in dataframes:
    print(type(df))

#Making sure each dataset was loaded at full size with all fields and columns. 
for df in dataframes:
    print(df.name)
    print(df.shape)
    print(df.info())
    print("--------\n")

11
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
Products
(77, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ProductID        77 non-null     int64  
 1   ProductName      77 non-null     object 
 2   SupplierID       77 non-null     int64  
 3   CategoryID       77 non-null     int64  
 4   QuantityPerUnit  77 non-null     object 
 5   UnitPrice        77 non-null     float64
 6   UnitsInStock     77 non-null     int64  
 7   UnitsOnOrder     77 non-null     int64 

In [ ]:
#Isolating each dataset 

Products = dataframes[0]
Suppliers = dataframes[1]
OrderDetails = dataframes[2]
Categories = dataframes[3]
Territories = dataframes[4]
Region = dataframes[5]
EmployeeTerritories = dataframes[6]
Employees = dataframes[7]
Orders = dataframes[8]
Shippers = dataframes[9]
Customers = dataframes[10]


pandas.core.frame.DataFrame

In [ ]:
#Data Cleaning and Preprocessing 

#Renaming [Order Details] both on the isolated dataframe and on the dataframes list
OrderDetails.name = 'OrderDetails'
dataframes[2].name = 'OrderDetails'

#Renaming UnitPrice column in Products to improve clarity.

Products['UnitPrice'].rename('UnitCost')

0      9.000
1      9.500
2      5.000
3     11.000
4     10.675
       ...  
72     7.500
73     5.000
74     3.875
75     9.000
76     6.500
Name: UnitCost, Length: 77, dtype: float64

Products
Suppliers
OrderDetails
Categories
Territories
Region
EmployeeTerritories
Employees
Orders
Shippers
Customers
